<a href="https://colab.research.google.com/github/YeabsiraNigusse/machine-Learning/blob/main/Decision%20Tree/Decision%20Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Decision Tree

In this notebook we will see
- what decision tree is
- what entropy is and how it is calcuated
- what information gain is and how we can caluculate it

A decision tree is a type of supervised machine learning used to categorize or make predictions based on how a previous set of questions were answered. The model is a form of supervised learning, meaning that the model is trained and tested on a set of data that contains the desired categorization.

In the context of Decision Trees, entropy is a measure of disorder or impurity in a node. Thus, a node with more variable composition, such as 2Pass and 2 Fail would be considered to have higher Entropy than a node which has only pass or only fail.

The entropy, defined as

$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$

In [1]:
!pip install pandas numpy matplotlib

In [5]:
from google.colab import files
uploaded = files.upload()

Saving deeplearning.mplstyle to deeplearning (1).mplstyle
Saving utils.py to utils (1).py


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import *

|                                                     |   Ear Shape | Face Shape | Whiskers |   Cat  |
|:---------------------------------------------------:|:---------:|:-----------:|:---------:|:------:|
| <img src="https://github.com/YeabsiraNigusse/machine-Learning/blob/main/Decision%20Tree/images/0.png?raw=1" alt="drawing" width="50"/> |   Pointy   |   Round     |  Present  |    1   |
| <img src="https://github.com/YeabsiraNigusse/machine-Learning/blob/main/Decision%20Tree/images/1.png?raw=1" alt="drawing" width="50"/> |   Floppy   |  Not Round  |  Present  |    1   |
| <img src="https://github.com/YeabsiraNigusse/machine-Learning/blob/main/Decision%20Tree/images/2.png?raw=1" alt="drawing" width="50"/> |   Floppy   |  Round      |  Absent   |    0   |
| <img src="https://github.com/YeabsiraNigusse/machine-Learning/blob/main/Decision%20Tree/images/3.png?raw=1" alt="drawing" width="50"/> |   Pointy   |  Not Round  |  Present  |    0   |
| <img src="https://github.com/YeabsiraNigusse/machine-Learning/blob/main/Decision%20Tree/images/4.png?raw=1" alt="drawing" width="50"/> |   Pointy   |   Round     |  Present  |    1   |
| <img src="https://github.com/YeabsiraNigusse/machine-Learning/blob/main/Decision%20Tree/images/5.png?raw=1" alt="drawing" width="50"/> |   Pointy   |   Round     |  Absent   |    1   |
| <img src="https://github.com/YeabsiraNigusse/machine-Learning/blob/main/Decision%20Tree/images/6.png?raw=1" alt="drawing" width="50"/> |   Floppy   |  Not Round  |  Absent   |    0   |
| <img src="https://github.com/YeabsiraNigusse/machine-Learning/blob/main/Decision%20Tree/images/7.png?raw=1" alt="drawing" width="50"/> |   Pointy   |  Round      |  Absent   |    1   |
| <img src="https://github.com/YeabsiraNigusse/machine-Learning/blob/main/Decision%20Tree/images/8.png?raw=1" alt="drawing" width="50"/> |    Floppy  |   Round     |  Absent   |    0   |
| <img src="https://github.com/YeabsiraNigusse/machine-Learning/blob/main/Decision%20Tree/images/9.png?raw=1" alt="drawing" width="50"/> |   Floppy   |  Round      |  Absent   |    0   |


We will use **one-hot encoding** to encode the categorical features. They will be as follows:

- Ear Shape: Pointy = 1, Floppy = 0
- Face Shape: Round = 1, Not Round = 0
- Whiskers: Present = 1, Absent = 0

Therefore, we have two sets:

- `X_train`: for each example, contains 3 features:
            - Ear Shape (1 if pointy, 0 otherwise)
            - Face Shape (1 if round, 0 otherwise)
            - Whiskers (1 if present, 0 otherwise)
            
- `y_train`: whether the animal is a cat
            - 1 if the animal is a cat
            - 0 otherwise

The data set as a collection of array

In [8]:
X_train = np.array([[1, 1, 1],
[0, 0, 1],
 [0, 1, 0],
 [1, 0, 1],
 [1, 1, 1],
 [1, 1, 0],
 [0, 0, 0],
 [1, 1, 0],
 [0, 1, 0],
 [0, 1, 0]])

y_train = np.array([1, 1, 0, 0, 1, 1, 0, 1, 0, 0])

### Entropy

In [9]:
def entropy(p):
    if p == 0 or p == 1:
        return 0
    else:
        return -p * np.log2(p) - (1- p)*np.log2(1 - p)

print(entropy(0.5))

1.0


The function help us to separate the data set based on its feature

e.g for ears it might be pointy or floppy

In [10]:
def split_indices(X, index_feature):
    """Given a dataset and a index feature, return two lists for the two split nodes, the left node has the animals that have
    that feature = 1 and the right node those that have the feature = 0
    index feature = 0 => ear shape
    index feature = 1 => face shape
    index feature = 2 => whiskers
    """
    left_indices = []
    right_indices = []
    for i,x in enumerate(X):
        if x[index_feature] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)
    return left_indices, right_indices

we calculate the two catagory entropy to test how match the feature perform well from other similar features. we take the average of both its left and right by considering their weight(total number of the catagory/ total dataset) and calculatating entropy for each `trget value` it might be cat or dog in our example.

In [11]:
def weighted_entropy(X,y,left_indices,right_indices):
    """
    This function takes the splitted dataset, the indices we choose to split and returns the weighted entropy.
    """
    w_left = len(left_indices)/len(X)
    w_right = len(right_indices)/len(X)
    p_left = sum(y[left_indices])/len(left_indices)
    p_right = sum(y[right_indices])/len(right_indices)

    weighted_entropy = w_left * entropy(p_left) + w_right * entropy(p_right)
    return weighted_entropy

In [12]:
left_indices, right_indices = split_indices(X_train, 0)
weighted_entropy(X_train, y_train, left_indices, right_indices)

0.7219280948873623

### Information Gain

the main thing to distingushe the performance of each feature in our dataset is by using entropy however the exact value we use to decide that which feature perform well is called `information Gain`

it is defined as
$$\text{Information Gain} = H(p_1^\text{node})- \left(w^{\text{left}}H\left(p_1^\text{left}\right) + w^{\text{right}}H\left(p_1^\text{right}\right)\right),$$


In [16]:
def information_gain(X, y, left_indices, right_indices):
    """
    Here, X has the elements in the node and y is theirs respectives classes
    """
    p_node = sum(y)/len(y)
    h_node = entropy(p_node)
    w_entropy = weighted_entropy(X,y,left_indices,right_indices)
    return h_node - w_entropy

In [14]:
information_gain(X_train, y_train, left_indices, right_indices)

0.2780719051126377

#### We can calculate the `information gain` for each feature and see how each feature well in reducing entropy on the classification problem.

In [15]:
for i, feature_name in enumerate(['Ear Shape', 'Face Shape', 'Whiskers']):
    left_indices, right_indices = split_indices(X_train, i)
    i_gain = information_gain(X_train, y_train, left_indices, right_indices)
    print(f"Feature: {feature_name}, information gain if we split the root node using this feature: {i_gain:.2f}")


Feature: Ear Shape, information gain if we split the root node using this feature: 0.28
Feature: Face Shape, information gain if we split the root node using this feature: 0.03
Feature: Whiskers, information gain if we split the root node using this feature: 0.12


The process is **recursive**, which means we must perform these calculations for each node until we meet a stopping criteria:

- If the tree depth after splitting exceeds a threshold
- If the resulting node has only 1 class
- If the information gain of splitting is below a threshold